# Category Composition Pie Chart

This notebook generates and exports a pie chart showing the composition of a category at a location

In [10]:
%run ../pathutils.ipynb
%run ../definitions.ipynb
%run ../database.ipynb
%run ../export.ipynb
%run pathutils.ipynb

In [11]:

import matplotlib.pyplot as plt

def export_composition_chart(export_folder_path, data, category):
    # Prepare the data
    labels = data.index
    sizes = data.values
    percentages = sizes / sizes.sum() * 100
    legend_labels = [f"{label} ({pct:.1f}%)" for label, pct in zip(labels, percentages)]

    # Create figure with extra vertical space for legend
    fig, ax = plt.subplots(figsize=(8, 8))

    wedges, _ = ax.pie(sizes, startangle=90, counterclock=False)

    # Equal aspect ratio
    ax.set_aspect('equal')
    ax.set_title(f"Composition of '{category}' Category by Species")

    # Remove tight_layout, manually control layout instead
    fig.subplots_adjust(bottom=0.3)  # Increase this if needed

    # Place legend fully outside the plot
    ax.legend(wedges, legend_labels,
                title="Species",
                loc='upper center',
                bbox_to_anchor=(0.5, -0.005),
                ncol=3,
                frameon=False)

    # Export the chart
    export_chart(export_folder_path, "composition", "png")

    # ShCloseow the plot
    plt.close(fig)

In [ ]:
country = None
location = None
category = None
year = None

In [ ]:
# Load the definitions
definitions = load_definitions("wildlife", "composition", country, location, category, year)

# Iterate over each definition to produce a report for each one
for definition in definitions:
    for year in range(int(definition["From"]), 1 + int(definition["To"])):
        # Construct the query
        query = construct_query("wildlife", "category_composition_chart.sql", {
            "LOCATION": definition["Location"],
            "CATEGORY": definition["Category"]
        })

        try:
            # Run the query to load the data - this raises a ValueError if there's no data
            df = query_data("wildlife", query)

            # Group by species and sum number of individuals
            species_counts = df.groupby("Species")["Count"].sum()

            # Sort values (optional, for nicer plots)
            species_counts = species_counts.sort_values(ascending=False)

            # Export the data to Excel
            export_folder_path = get_category_composition_folder_path(definition["Country"], definition["Location"], definition["Category"])
            export_to_spreadsheet(export_folder_path, "composition.xlsx", { "Category Composition": species_counts })

            # Export the chart
            export_composition_chart(export_folder_path, species_counts, definition["Category"])

        except ValueError:
            print(f"WARNING: No data found for location = {definition['Location']}, category = {definition['Category']}")